In [1]:
import importlib
import os
import re
import sys
from collections import defaultdict, namedtuple
from dataclasses import dataclass, fields, asdict
from pathlib import Path

import nibabel as nib
import numpy as np
import platform

In [3]:
from mri_preproc.paths import data_file_manager as datafm
from mri_preproc.paths import hemond_data as hd
from mri_preproc.paths.record import Record

In [220]:
importlib.reload(datafm)
importlib.reload(hd)
importlib.reload(sys.modules['record']).DataSet

record.DataSet

### Choroid Plexus Dataset

In [4]:
from pathlib import Path
data_dir = Path("/mnt/e/Data/Hemond/flair")
check = list(data_dir.glob("sub*"))

## Hemond Data

In [221]:
data_dir = Path("/mnt/e/Data/Hemond/flair")
dataset = hd.scan_data_dir(data_dir)

/home/srs-9/Dev/ms_mri/monai/preproc/hemond_data.py:64: UserWarning: No scan exists for subject: 1492 and session: 20190620
  return self._scans


### Assign data to train and test

In [237]:
import random

scans_no_label = []
for i, scan in enumerate(dataset):
    if not scan.has_label():
        scans_no_label.append(i)

fraction_ts = 0.1
n_scans = len(dataset)
n_ts = int(fraction_ts * n_scans)
inds = [i for i in range(n_scans)]
random.shuffle(inds)

for i in scans_no_label:
    inds.remove(i)
    inds.insert(0, i)

for i in inds[:n_ts]:
    dataset[i].cond = 'ts'
for i in inds[n_ts:]:
    dataset[i].cond = 'tr'

In [236]:
len(dataset)

356

### Inspect Data

In [196]:
def load_scan(scan_path: Path) -> np.ndarray:
    img = nib.load(scan_path)
    return np.array(img.dataobj)

In [203]:
image = load_scan(dataset[10].image)
image.shape

(198, 300, 300)

## Brats Data

Data for 3D autoseg

### Inspect Data

In [1]:
def load_scan(scan_path: Path) -> np.ndarray:
    img = nib.load(scan_path)
    return np.array(img.dataobj)

NameError: name 'Path' is not defined

In [ ]:
data_dir = "/mnt/t/Data/Task01_BrainTumor"

## Lesjak Data

In [ ]:
basepath = Path("/mnt/f/Data/ms_mri/lesjak_2017")
data_dir = basepath / "data"

lesjak_datafm = datafm.LesjakData(data_dir)
subjects = lesjak_datafm.subjects
modalities = ["FLAIR", "T1W", "T1WKS", "T2W"]
lesjak_datafm.get_scan(subjects[0], modalities[1])

In [ ]:
def load_scan(scan_path: Path) -> np.ndarray:
    img = nib.load(scan_path)
    return np.array(img.dataobj)

In [ ]:
subject = subjects[1]
imgs = [nib.load(lesjak_datafm.get_scan(subject, mod)) for mod in modalities]

In [ ]:
imgs[3].affine

In [ ]:
scan_paths = [lesjak_datafm.get_scan(subject, mod) for mod in modalities]
scans = [load_scan(path) for path in scan_paths]

In [ ]:
labels = datafm.assign_train_test(subjects, 0.2)

In [ ]:
basepath = Path("/mnt/f/Data/ms_mri/Brats_3d_seg/Task01_BrainTumour")
imagesTr = basepath / "imagesTr"
image_files = os.scandir(imagesTr)
image_paths = [file.path for file in image_files if "._" not in file.name]

In [ ]:
img = nib.load(image_paths[0])

a = np.array(img.dataobj)

In [ ]:
test = tuple()
if not test:
    print("uninitialzed")

In [ ]:
import random
fraction_ts = 0.2

n_subj = len(subjects)
n_ts = int(n_subj * fraction_ts)

random.shuffle(subjects)
subj_ts = [(subj, "ts") for subj in subjects[:n_ts]]
subj_tr = [(subj, "tr") for subj in subjects[n_ts:]]

subj_ts + subj_tr